# 目的
stella_en_1.5B_v5をfine-tuningする

ref: https://sbert.net/docs/sentence_transformer/training_overview.html#trainer

In [1]:
!nvidia-smi

Mon Nov  4 09:19:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               On  | 00000000:01:00.0 Off |                  Off |
| 32%   39C    P8              19W / 300W |  48658MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Setting

In [ ]:
EXP_NAME = "e016-ret-stella"
DATA_PATH = "data"
MODEL_NAME = "dunzhang/stella_en_1.5B_v5"
ENV_PATH = "env_file"
COMPETITION_NAME = "eedi-mining-misconceptions-in-mathematics"
RETRIVED_FILE_PATH = (
    "output/retriever/e003-ret-bge/e003-ret-bge-ret25-map0.1841-recall0.5506.csv"
)

# RETRIVED_EX_FILE_PATH = "output/retriever/e013-ret-bge-prepare/e013-ret-bge-prepare-ret25-map0.4374-recall0.8410.csv"

DATASET_NAME = EXP_NAME
OUTPUT_PATH = f"output/retriever/{EXP_NAME}"
# MODEL_OUTPUT_PATH = f"{OUTPUT_PATH}/trained_model"
MODEL_OUTPUT_PATH = f"{OUTPUT_PATH}/checkpoint-5010"

TRAIN_USE_NUM = 5
RETRIEVE_NUM = 25  # TODO: 多くしてみる

EPOCH = 5
BS = 1  # 2  # 4  # 8
GRAD_ACC_STEP = 1  # bugがあるので1にする。# 128 // BS
LR = 2e-5

TRAINING = True
DEBUG = False
WANDB = True

In [3]:
def resolve_path(base_path: str) -> str:
    import os

    cwd = os.getcwd()
    print(cwd)
    if cwd == f"/notebooks":
        print("Jupyter Kernel By VSCode!")
        return f"/notebooks/{COMPETITION_NAME}/{base_path}"
    elif cwd == f"/notebooks/{COMPETITION_NAME}":
        print("nohup!")
        return base_path
    elif cwd == f"/notebooks/{COMPETITION_NAME}/{COMPETITION_NAME}/exp":
        print("Jupyter Lab!")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/reranker":
        print("VastAi! Reranker")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/retriever":
        print("VastAi! Retriever")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}":
        print("VastAi!")
        return base_path
    else:
        raise Exception("Unknown environment")


DATA_PATH = resolve_path(DATA_PATH)
print(DATA_PATH)
OUTPUT_PATH = resolve_path(OUTPUT_PATH)
print(OUTPUT_PATH)
MODEL_OUTPUT_PATH = resolve_path(MODEL_OUTPUT_PATH)
print(MODEL_OUTPUT_PATH)
ENV_PATH = resolve_path(ENV_PATH)
print(ENV_PATH)
RETRIVED_FILE_PATH = resolve_path(RETRIVED_FILE_PATH)
print(RETRIVED_FILE_PATH)
# RETRIVED_EX_FILE_PATH = resolve_path(RETRIVED_EX_FILE_PATH)
# print(RETRIVED_EX_FILE_PATH)

/root/eedi-mining-misconceptions-in-mathematics/exp/retriever
VastAi! Retriever
../../data
/root/eedi-mining-misconceptions-in-mathematics/exp/retriever
VastAi! Retriever
../../output/retriever/e016-ret-stella
/root/eedi-mining-misconceptions-in-mathematics/exp/retriever
VastAi! Retriever
../../output/retriever/e016-ret-stella/checkpoint-5010
/root/eedi-mining-misconceptions-in-mathematics/exp/retriever
VastAi! Retriever
../../env_file
/root/eedi-mining-misconceptions-in-mathematics/exp/retriever
VastAi! Retriever
../../output/retriever/e003-ret-bge/e003-ret-bge-ret25-map0.1841-recall0.5506.csv


# Import

In [4]:
import os
import numpy as np

from datasets import load_dataset, Dataset

import wandb
import polars as pl

from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
)
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator

In [5]:
import transformers
import sentence_transformers

assert transformers.__version__ == "4.44.2"
assert sentence_transformers.__version__ == "3.1.0"

In [6]:
NUM_PROC = 16

# Setting

In [7]:
from dotenv import load_dotenv

load_dotenv(f"{ENV_PATH}/.env")

True

# WANDB

In [8]:
if WANDB:
    wandb.login(key=os.environ["WANDB_API_KEY"])
    wandb.init(project=COMPETITION_NAME, name=EXP_NAME)
    REPORT_TO = "wandb"
else:
    REPORT_TO = "none"

REPORT_TO

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sinchir0. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


'wandb'

# Data Load

In [9]:
train = (
    load_dataset(
        "csv",
        data_files=RETRIVED_FILE_PATH,
        split="train",
    )
    .filter(
        lambda example: example["MisconceptionId"] is not None,
        num_proc=NUM_PROC,
    )
    .filter(  # anchor、positive、negativeの構成にするため、positiveとnegativeが一致している行を削除する
        lambda example: example["MisconceptionId"] != example["PredictMisconceptionId"],
        num_proc=NUM_PROC,
    )
)

In [10]:
train

Dataset({
    features: ['QuestionId', 'ConstructName', 'SubjectName', 'QuestionText', 'CorrectAnswer', 'AnswerType', 'AnswerText', 'AllText', 'AnswerAlphabet', 'QuestionId_Answer', 'MisconceptionId', 'PredictMisconceptionId', 'target', 'MisconceptionName', 'PredictMisconceptionName'],
    num_rows: 106844
})

In [11]:
# # 同じMisconceptionNameのpositiveの例を25個→5個に減らす
# train = Dataset.from_pandas(
#     train.to_pandas().groupby("QuestionId_Answer").head(TRAIN_USE_NUM)
# )

In [12]:
if DEBUG:
    train = train.select(range(1000))

In [13]:
print(train["AllText"][0])

Use the order of operations to carry out calculations involving powers BIDMAS \[
3 \times 2+4-5
\]
Where do the brackets need to go to make the answer equal \( 13 \) ? Does not need brackets


In [14]:
print(train["MisconceptionName"][0])

Confuses the order of operations, believes addition comes before multiplication 


In [15]:
print(train["PredictMisconceptionName"][0])

Applies BIDMAS in strict order (does not realize addition and subtraction, and multiplication and division, are of equal priority)


In [16]:
# # load ex data
# train_ex = (
#     load_dataset(
#         "csv",
#         data_files=RETRIVED_EX_FILE_PATH,
#         split="train",
#     )
#     .filter(  # Nameを正しく生成できず、Idが紐づかなかったデータを落とす
#         lambda example: example["MisconceptionId"] is not None,
#         num_proc=NUM_PROC,
#     )
#     .filter(  # anchor、positive、negativeの構成にするため、positiveとnegativeが一致している行を削除する
#         lambda example: example["MisconceptionId"] != example["PredictMisconceptionId"],
#         num_proc=NUM_PROC,
#     )
# )

# train_ex

# Train Valid Split

In [17]:
train, valid = (
    train.filter(lambda example: example["QuestionId"] % 3 != 0, num_proc=NUM_PROC),
    train.filter(lambda example: example["QuestionId"] % 3 == 0, num_proc=NUM_PROC),
)

In [18]:
print(train)
print(valid)

Dataset({
    features: ['QuestionId', 'ConstructName', 'SubjectName', 'QuestionText', 'CorrectAnswer', 'AnswerType', 'AnswerText', 'AllText', 'AnswerAlphabet', 'QuestionId_Answer', 'MisconceptionId', 'PredictMisconceptionId', 'target', 'MisconceptionName', 'PredictMisconceptionName'],
    num_rows: 71119
})
Dataset({
    features: ['QuestionId', 'ConstructName', 'SubjectName', 'QuestionText', 'CorrectAnswer', 'AnswerType', 'AnswerText', 'AllText', 'AnswerAlphabet', 'QuestionId_Answer', 'MisconceptionId', 'PredictMisconceptionId', 'target', 'MisconceptionName', 'PredictMisconceptionName'],
    num_rows: 35725
})


In [19]:
# from datasets import concatenate_datasets

# train = concatenate_datasets([train, train_ex])

In [20]:
# print(train)
# print(valid)

# Fine-tuning BGE

In [21]:
if TRAINING:
    model = SentenceTransformer(MODEL_NAME, trust_remote_code=True)

    loss = MultipleNegativesRankingLoss(model)

    args = SentenceTransformerTrainingArguments(
        # Required parameter:
        output_dir=OUTPUT_PATH,
        # Optional training parameters:
        num_train_epochs=EPOCH,
        per_device_train_batch_size=BS,  # 16,
        gradient_accumulation_steps=GRAD_ACC_STEP,
        per_device_eval_batch_size=BS,  # 16,
        eval_accumulation_steps=GRAD_ACC_STEP,
        learning_rate=LR,
        weight_decay=0.01,
        warmup_ratio=0.1,
        fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
        fp16_full_eval=True,
        bf16=False,  # Set to True if you have a GPU that supports BF16
        batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
        # Optional tracking/debugging parameters:
        lr_scheduler_type="cosine_with_restarts",
        eval_strategy="steps",
        eval_steps=0.1,
        save_strategy="steps",
        save_steps=0.1,
        save_total_limit=2,
        logging_steps=100,
        report_to=REPORT_TO,  # Will be used in W&B if `wandb` is installed
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
    )

    dev_evaluator = TripletEvaluator(
        anchors=valid["AllText"],
        positives=valid["MisconceptionName"],
        negatives=valid["PredictMisconceptionName"],
        name=f"{MODEL_NAME}-dev",
    )
    # dev_evaluator(model)

    trainer = SentenceTransformerTrainer(
        model=model,
        args=args,
        train_dataset=train.select_columns(
            ["AllText", "MisconceptionName", "PredictMisconceptionName"]
        ),
        eval_dataset=valid.select_columns(
            ["AllText", "MisconceptionName", "PredictMisconceptionName"]
        ),
        loss=loss,
        evaluator=dev_evaluator,
    )

    trainer.train()
    model.save_pretrained(MODEL_OUTPUT_PATH)
else:
    model = SentenceTransformer(MODEL_OUTPUT_PATH)

modules.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/174k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

modeling_qwen.py:   0%|          | 0.00/65.2k [00:00<?, ?B/s]

ImportError: This modeling file requires the following packages that were not found in your environment: flash_attn. Run `pip install flash_attn`

# Make Vector

In [ ]:
common_col = [
    "QuestionId",
    "ConstructName",
    "SubjectName",
    "QuestionText",
    "CorrectAnswer",
]

train_long = (
    pl.read_csv(f"{DATA_PATH}/train.csv")
    .select(
        pl.col(common_col + [f"Answer{alpha}Text" for alpha in ["A", "B", "C", "D"]])
    )
    .unpivot(
        index=common_col,
        variable_name="AnswerType",
        value_name="AnswerText",
    )
    .with_columns(
        pl.concat_str(
            [
                pl.col("ConstructName"),
                pl.col("SubjectName"),
                pl.col("QuestionText"),
                pl.col("AnswerText"),
            ],
            separator=" ",
        ).alias("AllText"),
        pl.col("AnswerType").str.extract(r"Answer([A-D])Text$").alias("AnswerAlphabet"),
    )
    .with_columns(
        pl.concat_str(
            [pl.col("QuestionId"), pl.col("AnswerAlphabet")], separator="_"
        ).alias("QuestionId_Answer"),
    )
    .sort("QuestionId_Answer")
)
train_long.head()

In [ ]:
train_misconception_long = (
    pl.read_csv(f"{DATA_PATH}/train.csv")
    .select(
        pl.col(
            common_col + [f"Misconception{alpha}Id" for alpha in ["A", "B", "C", "D"]]
        )
    )
    .unpivot(
        index=common_col,
        variable_name="MisconceptionType",
        value_name="MisconceptionId",
    )
    .with_columns(
        pl.col("MisconceptionType")
        .str.extract(r"Misconception([A-D])Id$")
        .alias("AnswerAlphabet"),
    )
    .with_columns(
        pl.concat_str(
            [pl.col("QuestionId"), pl.col("AnswerAlphabet")], separator="_"
        ).alias("QuestionId_Answer"),
    )
    .sort("QuestionId_Answer")
    .select(pl.col(["QuestionId_Answer", "MisconceptionId"]))
    .with_columns(pl.col("MisconceptionId").cast(pl.Int64))
)
train_misconception_long.head()

In [ ]:
# join MisconceptionId
train_long = train_long.join(train_misconception_long, on="QuestionId_Answer")

In [ ]:
valid_long = train_long.filter(
    pl.col("QuestionId_Answer").is_in(set(valid["QuestionId_Answer"]))
)

In [ ]:
valid_long

In [ ]:
# model = SentenceTransformer(MODEL_OUTPUT_PATH)

valid_long_vec = model.encode(
    valid_long["AllText"].to_list(), normalize_embeddings=True
)
misconception_mapping = pl.read_csv(f"{DATA_PATH}/misconception_mapping.csv")
misconception_mapping_vec = model.encode(
    misconception_mapping["MisconceptionName"].to_list(), normalize_embeddings=True
)
print(valid_long_vec.shape)
print(misconception_mapping_vec.shape)

In [ ]:
# misconception_mapping_vecを保存する
os.makedirs(OUTPUT_PATH, exist_ok=True)
np.save(f"{OUTPUT_PATH}/misconception_mapping_vec.npy", misconception_mapping_vec)

In [ ]:
valid_cos_sim_arr = cosine_similarity(valid_long_vec, misconception_mapping_vec)
valid_sorted_indices = np.argsort(-valid_cos_sim_arr, axis=1)

In [ ]:
# example
def print_example(df: pl.DataFrame, sorted_indices: np.ndarray, idx: int) -> None:
    print(f"Query idx{idx}")
    print(df["AllText"][idx])
    print("\nCos Sim No.1")
    print(misconception_mapping["MisconceptionName"][int(sorted_indices[idx, 0])])
    print("\nCos Sim No.2")
    print(misconception_mapping["MisconceptionName"][int(sorted_indices[idx, 1])])

In [ ]:
print_example(train_long, valid_sorted_indices, 0)

In [ ]:
print_example(train_long, valid_sorted_indices, 1)

# Evaluate

In [ ]:
valid_long = valid_long.with_columns(
    pl.Series(valid_sorted_indices[:, :RETRIEVE_NUM].tolist()).alias(
        "PredictMisconceptionId"
    )
)

In [ ]:
# https://www.kaggle.com/code/cdeotte/how-to-train-open-book-model-part-1#MAP@3-Metric
def map_at_25(predictions, labels):
    map_sum = 0
    for x, y in zip(predictions, labels):
        z = [1 / i if y == j else 0 for i, j in zip(range(1, 26), x)]
        map_sum += np.sum(z)
    return map_sum / len(predictions)

In [ ]:
map_at_25_score = map_at_25(
    valid_long.filter(pl.col("MisconceptionId").is_not_null())[
        "PredictMisconceptionId"
    ],
    valid_long.filter(pl.col("MisconceptionId").is_not_null())["MisconceptionId"],
)
map_at_25_score

In [ ]:
def recall(predictions, labels):
    acc_num = np.sum([1 for x, y in zip(predictions, labels) if y in x])
    return acc_num / len(predictions)


recall_score = recall(
    valid_long.filter(pl.col("MisconceptionId").is_not_null())[
        "PredictMisconceptionId"
    ],
    valid_long.filter(pl.col("MisconceptionId").is_not_null())["MisconceptionId"],
)
recall_score

In [ ]:
# output_textを保存
with open(f"{MODEL_OUTPUT_PATH}/cv_score.txt", "w") as f:
    f.write(f"MAP@25:{map_at_25_score:.4f}, Recall:{recall_score:.4f}")

# Make Retrieved Train File

In [ ]:
valid_retrieved = (
    valid_long.filter(pl.col("MisconceptionId").is_not_null())
    .explode("PredictMisconceptionId")
    .with_columns(
        (pl.col("MisconceptionId") == pl.col("PredictMisconceptionId"))
        .cast(pl.Int64)
        .alias("target")
    )
    .join(
        misconception_mapping.with_columns(pl.all().name.prefix("Predict")),
        on="PredictMisconceptionId",
    )
)
valid_retrieved.head()

In [ ]:
valid_retrieved.write_csv(
    f"{MODEL_OUTPUT_PATH}/{EXP_NAME}-valid-ret{RETRIEVE_NUM}-map{map_at_25_score:.4f}-recall{recall_score:.4f}.csv",
)

In [ ]:
print(
    f"{MODEL_OUTPUT_PATH}/{EXP_NAME}-valid-ret{RETRIEVE_NUM}-map{map_at_25_score:.4f}-recall{recall_score:.4f}.csv",
)

# Kaggle Upload

In [ ]:
import os
import json

from kaggle.api.kaggle_api_extended import KaggleApi


def dataset_create_new(dataset_name: str, upload_dir: str):
    # if "_" in dataset_name:
    #     raise ValueError("datasetの名称に_の使用は禁止です")
    dataset_metadata = {}
    dataset_metadata["id"] = f"sinchir0/{dataset_name}"
    dataset_metadata["licenses"] = [{"name": "CC0-1.0"}]
    dataset_metadata["title"] = dataset_name
    with open(os.path.join(upload_dir, "dataset-metadata.json"), "w") as f:
        json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    api.dataset_create_new(folder=upload_dir, convert_to_csv=False, dir_mode="tar")


print(f"Create Dataset name:{DATASET_NAME}, output_dir:{OUTPUT_PATH}")
dataset_create_new(dataset_name=DATASET_NAME, upload_dir=OUTPUT_PATH)